# Algorytm symulowanego wyżarzania (SA)

Grupa: Amelia Madej, Justyna Sarkowicz, Olga Sieradzan, Weronika Duda i Aleksandra Węgrzyn

## Biblioteki

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
import math
from collections import deque
from openpyxl import load_workbook

## Pobór danych

In [2]:
file_path1 = "./Dane_TSP_48.xlsx"
file_path2 = "./Dane_TSP_76.xlsx"
file_path3 = "./Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()
data2 = pd.read_excel(file_path2, index_col=0).to_numpy()
data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Rodzaj ruchów

Poniżej znajduje się implementacja trzech rodzajów ruchów (generowania rozwiązań sąsiednich)

In [4]:
# Wybieramy dwie pozycje i zmieniamy je miejscami
def swap_move(path):  
    i, j = random.sample(range(len(path)), 2)
    new_path = path[:]
    new_path[i], new_path[j] = new_path[j], new_path[i]
    return new_path
 
# Odwracamy segment pomiędzy wybranymi wartościami czyli
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    i, j = sorted(random.sample(range(len(path)), 2))
    new_path = path[:i] + path[i:j][::-1] + path[j:]
    return new_path
 
# Wybieramy jeden wierzchołek w ścieżce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    i, j = random.sample(range(len(path)), 2)
    new_path = path[:]
    node = new_path.pop(i)
    new_path.insert(j, node)
    return new_path

## Długość ścieżki

In [5]:
# Funkcja obliczająca całkowity koszt ścieżki
# Suma odległości pomiędzy kolejnymi miastami oraz powrót do miasta początkowego

def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Implementacja algorytmu

In [6]:
def simulated_annealing(
    distance_matrix,
    initial_temperature=1000,
    cooling_rate=0.99,
    max_iterations=10000,
    min_temperature=1e-100,
    time_limit=None,
    move_type="two_opt"  
):
    start_time = time.time()
    n = len(distance_matrix)

    # Losowe wygenerowanie początkowej trasy
    initial_route = list(range(n))
    random.shuffle(initial_route)
    
    # Mapowanie typu ruchu na odpowiednią funkcję generatora sąsiadów
    move_generators = {
        "swap": swap_move,
        "two_opt": two_opt_move,
        "insertion": insertion_move
    }

    if move_type not in move_generators:
        raise ValueError(f"Nieznany typ ruchu: {move_type}")

    move_generator = move_generators[move_type]

    # Inicjalizacja bieżącego i najlepszego rozwiązania
    current_solution = initial_route
    current_cost = calculate_path_cost(distance_matrix, current_solution)
    best_solution = current_solution[:]
    best_cost = current_cost

    # Ustawienie początkowej temperatury
    temperature = initial_temperature

    for iteration in range(max_iterations):
        if time_limit and (time.time() - start_time > time_limit):
            break
        
        # Sprawdzenie, czy temperatura osiągnęła wartość minimalną
        if temperature < min_temperature:
            break

        # Generowanie sąsiada na podstawie wybranego typu ruchu
        neighbors = list(move_generator(current_solution))
        neighbor = random.choice(neighbors)
        neighbor_cost = calculate_path_cost(distance_matrix, neighbor)

        
        # Różnica w kosztach między bieżącym a sąsiednim rozwiązaniem
        delta = neighbor_cost - current_cost

        if delta < 0:
            current_solution = neighbor
            current_cost = neighbor_cost
        else:
            # Akceptacja gorszego rozwiązania z prawdopodobieństwem zależnym od temperatury
            acceptance_probability = math.exp(-delta / temperature)
            if random.random() < acceptance_probability:
                current_solution = neighbor
                current_cost = neighbor_cost


        # Aktualizacja najlepszego rozwiązania, jeśli obecne jest lepsze
        if current_cost < best_cost:
            best_solution = current_solution[:]
            best_cost = current_cost

        # Schładzanie temperatury
        temperature *= cooling_rate

        total_time = time.time() - start_time

    return best_solution, best_cost, total_time

## Generowanie rozwiązań

Poniżej znajduje się zestawienie wyników z uwzględnieniem wpływu na wyniki różnych wartości parametrów algorytmu

Za podstawowe dane przyjmujemy :

* initial_temperature = 10000

* cooling_rate = 0.99

* max_iterations = 10000

* min_temperature = 1e-100

* time_limit = `None`

* move_type = `two_opt`


In [7]:
def run_multiple_tests(data, dataset_name, num_tests=10):
    results = []
    for _ in range(num_tests):
        best_path, best_cost, best_time = simulated_annealing(data)
        results.append({
            "WYNIK": best_cost,
            "ŚCIEŻKA": best_path,
            "CZAS": best_time,
            "DATASET": dataset_name
        })
    return results

In [7]:
all_results = []
all_results.extend(run_multiple_tests(data1, "DATA1"))
all_results.extend(run_multiple_tests(data2, "DATA2"))
all_results.extend(run_multiple_tests(data3, "DATA3"))

basic = pd.DataFrame(data = all_results)

with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='w') as writer:
    basic.to_excel(writer, sheet_name = "Basic", index=False)

Iteracja 0, najlepszy koszt: 50903, temperatura: 990.00
Iteracja 100, najlepszy koszt: 32307, temperatura: 362.37
Iteracja 200, najlepszy koszt: 25980, temperatura: 132.64
Iteracja 300, najlepszy koszt: 23411, temperatura: 48.55
Iteracja 400, najlepszy koszt: 20616, temperatura: 17.77
Iteracja 500, najlepszy koszt: 18795, temperatura: 6.50
Iteracja 600, najlepszy koszt: 18016, temperatura: 2.38
Iteracja 700, najlepszy koszt: 16409, temperatura: 0.87
Iteracja 800, najlepszy koszt: 15641, temperatura: 0.32
Iteracja 900, najlepszy koszt: 15352, temperatura: 0.12
Iteracja 1000, najlepszy koszt: 14430, temperatura: 0.04
Iteracja 1100, najlepszy koszt: 14165, temperatura: 0.02
Iteracja 1200, najlepszy koszt: 14084, temperatura: 0.01
Iteracja 1300, najlepszy koszt: 13773, temperatura: 0.00
Iteracja 1400, najlepszy koszt: 12828, temperatura: 0.00
Iteracja 1500, najlepszy koszt: 12820, temperatura: 0.00
Iteracja 1600, najlepszy koszt: 12673, temperatura: 0.00
Iteracja 1700, najlepszy koszt: 126

Badanie wpływu parametru *move_type*

In [8]:
def test_move_type(data, dataset_name, num_repeats=10):
    results = []
    move_types = ["swap", "two_opt", "insertion"]
    for move_type in move_types:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = simulated_annealing(data, move_type=move_type)
            results.append({"PARAMETR": move_type, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_move_type_1 = test_move_type(data1, "DATA1")
df_move_type_2 = test_move_type(data2, "DATA2")
df_move_type_3 = test_move_type(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
neighborhood_type_results = pd.concat([df_move_type_1, df_move_type_2, df_move_type_3], ignore_index=True)
with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='a') as writer:
    neighborhood_type_results.to_excel(writer, sheet_name="move_type", index=False)

Iteracja 0, najlepszy koszt: 50856, temperatura: 990.00
Iteracja 100, najlepszy koszt: 33543, temperatura: 362.37
Iteracja 200, najlepszy koszt: 30394, temperatura: 132.64
Iteracja 300, najlepszy koszt: 28157, temperatura: 48.55
Iteracja 400, najlepszy koszt: 26150, temperatura: 17.77
Iteracja 500, najlepszy koszt: 25560, temperatura: 6.50
Iteracja 600, najlepszy koszt: 24024, temperatura: 2.38
Iteracja 700, najlepszy koszt: 22770, temperatura: 0.87
Iteracja 800, najlepszy koszt: 22389, temperatura: 0.32
Iteracja 900, najlepszy koszt: 21834, temperatura: 0.12
Iteracja 1000, najlepszy koszt: 21602, temperatura: 0.04
Iteracja 1100, najlepszy koszt: 21285, temperatura: 0.02
Iteracja 1200, najlepszy koszt: 20755, temperatura: 0.01
Iteracja 1300, najlepszy koszt: 20755, temperatura: 0.00
Iteracja 1400, najlepszy koszt: 20601, temperatura: 0.00
Iteracja 1500, najlepszy koszt: 20601, temperatura: 0.00
Iteracja 1600, najlepszy koszt: 20370, temperatura: 0.00
Iteracja 1700, najlepszy koszt: 202

Badanie wpływu zmian wartości parametru *initial_temperature*

In [10]:
def test_initial_temperature(data, dataset_name, num_repeats=10):
    results = []
    initial_temperatures = [1000, 3000, 5000, 10000, 30000, 50000, 100000]
    for initial_temperature in initial_temperatures:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = simulated_annealing(data, initial_temperature=initial_temperature)
            results.append({"PARAMETR": initial_temperature, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_initial_temperature_1 = test_initial_temperature(data1, "DATA1")
df_initial_temperature_2 = test_initial_temperature(data2, "DATA2")
df_initial_temperature_3 = test_initial_temperature(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
num_starts = pd.concat([df_initial_temperature_1, df_initial_temperature_2, df_initial_temperature_3], ignore_index=True)
with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='a') as writer:
    num_starts.to_excel(writer, sheet_name = "initial_temperature", index=False)

Iteracja 0, najlepszy koszt: 49234, temperatura: 990.00
Iteracja 100, najlepszy koszt: 31327, temperatura: 362.37
Iteracja 200, najlepszy koszt: 26224, temperatura: 132.64
Iteracja 300, najlepszy koszt: 22240, temperatura: 48.55
Iteracja 400, najlepszy koszt: 21929, temperatura: 17.77
Iteracja 500, najlepszy koszt: 20178, temperatura: 6.50
Iteracja 600, najlepszy koszt: 17987, temperatura: 2.38
Iteracja 700, najlepszy koszt: 16804, temperatura: 0.87
Iteracja 800, najlepszy koszt: 16515, temperatura: 0.32
Iteracja 900, najlepszy koszt: 16174, temperatura: 0.12
Iteracja 1000, najlepszy koszt: 15774, temperatura: 0.04
Iteracja 1100, najlepszy koszt: 15150, temperatura: 0.02
Iteracja 1200, najlepszy koszt: 15150, temperatura: 0.01
Iteracja 1300, najlepszy koszt: 14693, temperatura: 0.00
Iteracja 1400, najlepszy koszt: 13782, temperatura: 0.00
Iteracja 1500, najlepszy koszt: 13362, temperatura: 0.00
Iteracja 1600, najlepszy koszt: 13362, temperatura: 0.00
Iteracja 1700, najlepszy koszt: 133

Badanie wpływu zmian wartości parametru *cooling_rate*

In [11]:
def test_cooling_rate(data, dataset_name, num_repeats=10):
    results = []
    for cooling_rate in np.arange(0.8, 0.99, 0.01):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = simulated_annealing(data, cooling_rate=cooling_rate)
            results.append({"PARAMETR": cooling_rate, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_cooling_rate_1 = test_cooling_rate(data1, "DATA1")
df_cooling_rate_2 = test_cooling_rate(data2, "DATA2")
df_cooling_rate_3 = test_cooling_rate(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
num_starts = pd.concat([df_cooling_rate_1, df_cooling_rate_2, df_cooling_rate_3], ignore_index=True)
with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='a') as writer:
    num_starts.to_excel(writer, sheet_name = "cooling_rate", index=False)

Iteracja 0, najlepszy koszt: 50837, temperatura: 800.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 55771, temperatura: 800.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 57531, temperatura: 800.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 56244, temperatura: 800.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 41451, temperatura: 800.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 49774, temperatura: 800.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 55109, temperatura: 800.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 43345, temperatura: 800.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 40878, temperatura: 800.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 42470, temperatura: 800.00
Temperatura osiągnęła wartość minimalną.
Iteracja 0, najlepszy koszt: 5

Badanie wpływu zmian wartości parametru *max_iterations*

In [13]:
def test_max_iterations(data, dataset_name, num_repeats=10):
    results = []
    for max_iteration in range(50, 500, 50):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = simulated_annealing(data, max_iterations=max_iteration)
            results.append({"PARAMETR": max_iteration, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_max_iterations_1 = test_max_iterations(data1, "DATA1")
df_max_iterations_2 = test_max_iterations(data2, "DATA2")
df_max_iterations_3 = test_max_iterations(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
num_starts = pd.concat([df_max_iterations_1, df_max_iterations_2, df_max_iterations_3], ignore_index=True)
with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='a') as writer:
    num_starts.to_excel(writer, sheet_name = "max_iterations", index=False)

Iteracja 0, najlepszy koszt: 46075, temperatura: 990.00
Iteracja 0, najlepszy koszt: 51810, temperatura: 990.00
Iteracja 0, najlepszy koszt: 42531, temperatura: 990.00
Iteracja 0, najlepszy koszt: 50162, temperatura: 990.00
Iteracja 0, najlepszy koszt: 52255, temperatura: 990.00
Iteracja 0, najlepszy koszt: 50270, temperatura: 990.00
Iteracja 0, najlepszy koszt: 51496, temperatura: 990.00
Iteracja 0, najlepszy koszt: 55010, temperatura: 990.00
Iteracja 0, najlepszy koszt: 51992, temperatura: 990.00
Iteracja 0, najlepszy koszt: 53926, temperatura: 990.00
Iteracja 0, najlepszy koszt: 58003, temperatura: 990.00
Iteracja 0, najlepszy koszt: 50734, temperatura: 990.00
Iteracja 0, najlepszy koszt: 53783, temperatura: 990.00
Iteracja 0, najlepszy koszt: 49651, temperatura: 990.00
Iteracja 0, najlepszy koszt: 54273, temperatura: 990.00
Iteracja 0, najlepszy koszt: 54193, temperatura: 990.00
Iteracja 0, najlepszy koszt: 51871, temperatura: 990.00
Iteracja 0, najlepszy koszt: 50429, temperatura:

Badanie wpływu zmian wartości parametru *min_temperature*

In [14]:
def test_min_temperature(data, dataset_name, num_repeats=10):
    results = []
    for min_temperature in np.logspace(-70, -10, 7, 10):
        for _ in range(num_repeats):
            best_path, best_cost, best_time = simulated_annealing(data, min_temperature=min_temperature)
            results.append({"PARAMETR": min_temperature, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_min_temperature_1 = test_min_temperature(data1, "DATA1")
df_min_temperature_2 = test_min_temperature(data2, "DATA2")
df_min_temperature_3 = test_min_temperature(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
num_starts = pd.concat([df_min_temperature_1, df_min_temperature_2, df_min_temperature_3], ignore_index=True)
with pd.ExcelWriter('SA.xlsx', engine='openpyxl', mode='a') as writer:
    num_starts.to_excel(writer, sheet_name = "min_temperature", index=False)

Iteracja 0, najlepszy koszt: 54216, temperatura: 990.00
Iteracja 100, najlepszy koszt: 42279, temperatura: 362.37
Iteracja 200, najlepszy koszt: 30948, temperatura: 132.64
Iteracja 300, najlepszy koszt: 24104, temperatura: 48.55
Iteracja 400, najlepszy koszt: 18681, temperatura: 17.77
Iteracja 500, najlepszy koszt: 17984, temperatura: 6.50
Iteracja 600, najlepszy koszt: 16867, temperatura: 2.38
Iteracja 700, najlepszy koszt: 16326, temperatura: 0.87
Iteracja 800, najlepszy koszt: 15736, temperatura: 0.32
Iteracja 900, najlepszy koszt: 15511, temperatura: 0.12
Iteracja 1000, najlepszy koszt: 14444, temperatura: 0.04
Iteracja 1100, najlepszy koszt: 14342, temperatura: 0.02
Iteracja 1200, najlepszy koszt: 14342, temperatura: 0.01
Iteracja 1300, najlepszy koszt: 13238, temperatura: 0.00
Iteracja 1400, najlepszy koszt: 13238, temperatura: 0.00
Iteracja 1500, najlepszy koszt: 13238, temperatura: 0.00
Iteracja 1600, najlepszy koszt: 13130, temperatura: 0.00
Iteracja 1700, najlepszy koszt: 124